In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Rotated square root cut
TR-025
:::
::::

# Rotating square root cuts

In [ ]:
%pip install -q ampform==0.14.8 ipywidgets==8.1.1 plotly==5.18.0 sympy==1.12

In [ ]:
%matplotlib widget
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math, display
from ipywidgets import FloatSlider, VBox, interactive_output

There are multiple solutions for $x$ to the equation $y^2 = x$. The fact that we usually take $y = \sqrt{x}$ with $\sqrt{-1} = i$ to be 'the' solution to this equation is just a matter of convention. It would be more complete to represent the solution as a set of points in the complex plane, that is, the set $S = \left\{\left(z, w\right)\in\mathbb{C}^2 | w^2=z\right\}$. This is set forms a **Riemann surface** in $\mathbb{C}^2$ space.

::::{margin}
:::{seealso} [Lecture 17](https://compwa.github.io/strong2020-salamanca/lecture17.html) on collision theory of the [STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803) by Miguel Albaladejo.
<!-- cspell:ignore Albaladejo -->
:::
::::

In the figure below we see the Riemann surface of a square root in $\mathbb{C}^2$&nbsp;space. The $xy$&nbsp;plane forms the complex domain $\mathbb{C}$, the $z$&nbsp;axis indicates the imaginary part of the Riemann surface and the color indicates the real part.

In [ ]:
resolution = 50
R, Θ = np.meshgrid(
    np.linspace(0, 1, num=resolution),
    np.linspace(-np.pi, +np.pi, num=resolution),
)
X = R * np.cos(Θ)
Y = R * np.sin(Θ)
Z = X + Y * 1j
T = np.sqrt(Z)
style = lambda t: dict(
    cmin=-1,
    cmax=+1,
    colorscale="RdBu_r",
    surfacecolor=t.real,
)
fig = go.Figure([
    go.Surface(x=X, y=Y, z=+T.imag, **style(+T), name="+√z"),
    go.Surface(x=X, y=Y, z=-T.imag, **style(-T), name="-√z", showscale=False),
])
fig.update_traces(selector=0, colorbar=dict(title="Re ±√z"))
fig.update_layout(
    height=600,
    scene=dict(
        xaxis_title="Re z",
        yaxis_title="Im z",
        zaxis_title="Im ±√z",
    ),
    title_text="Riemann surface of a square root",
    title_x=0.5,
)
fig.show()

In [ ]:
@unevaluated
class RotatedSqrt(sp.Expr):
    z: Any
    phi: Any = 0
    _latex_repr_ = R"\sqrt[{phi}]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z, phi = self.args
        return sp.exp(-phi * sp.I / 2) * sp.sqrt(z * sp.exp(phi * sp.I))


z, phi = sp.symbols("z phi")
expr = RotatedSqrt(z, phi)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
symbols = (z, phi)
func = sp.lambdify(symbols, expr.doit())

In [ ]:
mpl_fig, axes = plt.subplots(
    figsize=(6, 8.5),
    gridspec_kw=dict(
        height_ratios=[1, 2],
    ),
    nrows=2,
)
ax1, ax2 = axes
ax1.set_ylabel(f"${sp.latex(expr)}$")
ax2.set_ylabel("$\mathrm{Im}\,z$")
ax2.axhline(0, c="black", ls="dotted", zorder=99)
for ax in axes:
    ax.set_xlabel("$\mathrm{Re}\,z$")
    ax.set_xticks([-1, 0, +1])
ax2.set_yticks([-1, 0, +1])

data = None
x_mpl = np.linspace(-1, +1, num=400)
X_mpl, Y_mpl = np.meshgrid(x_mpl, x_mpl)
Z_mpl = X_mpl + Y_mpl * 1j

sliders = dict(
    phi=FloatSlider(
        min=-3 * np.pi,
        max=+3 * np.pi,
        step=np.pi / 8,
        description="phi",
    ),
)


def plot(phi):
    global data
    ax1.set_title(Rf"${sp.latex(expr)} \qquad \phi={phi / np.pi:.4g}\pi$")
    t_mpl = func(x_mpl, phi)
    T_mpl = func(Z_mpl, phi)
    if data is None:
        data = {
            "real": ax1.plot(x_mpl, t_mpl.real, label="real")[0],
            "imag": ax1.plot(x_mpl, t_mpl.imag, label="imag")[0],
            "2D": ax2.pcolormesh(X_mpl, Y_mpl, T_mpl.imag, cmap=plt.cm.coolwarm),
        }
    else:
        data["real"].set_ydata(t_mpl.real)
        data["imag"].set_ydata(t_mpl.imag)
        data["2D"].set_array(T_mpl.imag)
    data["2D"].set_clim(vmin=-1, vmax=+1)
    ax1.set_ylim(-1.2, +1.2)
    mpl_fig.canvas.draw_idle()


ui = VBox(tuple(sliders.values()))
output = interactive_output(plot, controls=sliders)
ax1.legend(loc="lower left")
cbar = plt.colorbar(data["2D"], ax=ax2, pad=0.02)
cbar.ax.set_yticks([-1, 0, +1])
cbar.ax.set_ylabel(f"${sp.latex(expr)}$")
mpl_fig.tight_layout()
display(ui, output)